In [1]:
import pandas as pd

import os
import glob

In [4]:
pwd

'/Users/filipeserro/Spiced_Projects/buynilla'

In [3]:
path = './data'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
os.chdir('..') # revert to main folder
print(result)

['friday.csv', 'monday.csv', 'thursday.csv', 'tuesday.csv', 'wednesday.csv']


In [23]:
df= pd.DataFrame()
for i in result:
    df_test = pd.read_csv(f'{i}', sep=';', index_col=0, parse_dates=True)
    df = pd.concat([df, df_test])
    df = df.sort_index()

In [24]:
df

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
2019-09-02 07:04:00,4,dairy
2019-09-02 07:04:00,5,spices
...,...,...
2019-09-06 21:50:00,1500,dairy
2019-09-06 21:50:00,1507,checkout
2019-09-06 21:50:00,1508,checkout


In [25]:
df['customer_ordered'] = df['customer_no'] #normally customer numbers start from 1 at each day! however, these are diffrent customers. therefore I created a new column customer_ordered. here all customers are unique!

for i in df.index.day.unique(): # go to each day 
    rowIndex = df.index[df.index.day > i] # receive the index numbers of each row higher than the day of concern
    df.loc[rowIndex,'customer_ordered'] = df.loc[rowIndex,'customer_ordered'] + df[df.index.day == i]['customer_no'].max() # incrementally add max customer number of a day to the day after it 
    

In [26]:
df = df.rename(columns={'customer_no': 'customer_no_original'})

In [27]:
df = df.rename(columns={'customer_ordered':'customer_no'})

In [28]:
df

,customer_no_original,location,customer_no
timestamp,,,
2019-09-02 07:03:00,1,dairy,1
2019-09-02 07:03:00,2,dairy,2
2019-09-02 07:04:00,3,dairy,3
2019-09-02 07:04:00,4,dairy,4
2019-09-02 07:04:00,5,spices,5
...,...,...,...
2019-09-06 21:50:00,1500,dairy,7435
2019-09-06 21:50:00,1507,checkout,7442
2019-09-06 21:50:00,1508,checkout,7443


1. Calculate the total number of customers in each section

In [29]:
df.groupby(['location']).count()

,customer_no_original,customer_no
location,,
checkout,7417,7417
dairy,4679,4679
drinks,3905,3905
fruit,5122,5122
spices,3754,3754


2. Calculate the total number of customers in each section over time

In [30]:
df.groupby([df.index, 'location']).count()

customer_no_original  customer_no
timestamp           location                                   
2019-09-02 07:03:00 dairy                        2            2
2019-09-02 07:04:00 dairy                        2            2
                    fruit                        1            1
                    spices                       3            3
2019-09-02 07:05:00 checkout                     2            2
...                                            ...          ...
2019-09-06 21:50:00 checkout                     2            2
                    dairy                        1            1
                    drinks                       1            1
                    fruit                        1            1
                    spices                       1            1

[13839 rows x 2 columns]

3. Display the number of customers at checkout over time

In [31]:
df_check = df[df['location'] == 'checkout']

In [32]:
df_check.groupby(df_check.index)['customer_no'].count()

timestamp
2019-09-02 07:05:00    2
2019-09-02 07:06:00    2
2019-09-02 07:07:00    1
2019-09-02 07:08:00    3
2019-09-02 07:09:00    1
                      ..
2019-09-06 21:37:00    1
2019-09-06 21:45:00    1
2019-09-06 21:47:00    2
2019-09-06 21:48:00    2
2019-09-06 21:50:00    2
Name: customer_no, Length: 3460, dtype: int64

4. Calculate the time each customer spent in the market

In [33]:
mylist = df.index.day.unique()
mylist

Int64Index([2, 3, 4, 5, 6], dtype='int64', name='timestamp')

In [34]:
df['day'] = df.index.day
df['day'] 

timestamp
2019-09-02 07:03:00    2
2019-09-02 07:03:00    2
2019-09-02 07:04:00    2
2019-09-02 07:04:00    2
2019-09-02 07:04:00    2
                      ..
2019-09-06 21:50:00    6
2019-09-06 21:50:00    6
2019-09-06 21:50:00    6
2019-09-06 21:50:00    6
2019-09-06 21:50:00    6
Name: day, Length: 24877, dtype: int64

In [35]:
df.head(3)

,customer_no_original,location,customer_no,day
timestamp,,,,
2019-09-02 07:03:00,1,dairy,1,2
2019-09-02 07:03:00,2,dairy,2,2
2019-09-02 07:04:00,3,dairy,3,2


In [36]:
#df_sub= df.loc[(df['customer_no']==1) & (df['day'] == 3)]
df_sub= df.loc[df['customer_no']==1]
df_sub

,customer_no_original,location,customer_no,day
timestamp,,,,
2019-09-02 07:03:00,1,dairy,1,2
2019-09-02 07:05:00,1,checkout,1,2


In [38]:
# old solution - wrong 
for i in df['customer_no_original'].unique():
    for j in df['day'].unique():
        df_sub = df.loc[(df['customer_no_original']==i) & (df['day'] == j)]
        time = df_sub.index.unique().max() - df_sub.index.unique().min() 
        print(f'customer No. {i}, at day {j} spend {time}')    

customer No. 1, at day 2 spend 0 days 00:02:00
customer No. 1, at day 3 spend 0 days 00:10:00
customer No. 1, at day 4 spend 0 days 00:02:00
customer No. 1, at day 5 spend 0 days 00:02:00
customer No. 1, at day 6 spend 0 days 00:05:00
customer No. 2, at day 2 spend 0 days 00:03:00
customer No. 2, at day 3 spend 0 days 00:15:00
customer No. 2, at day 4 spend 0 days 00:06:00
customer No. 2, at day 5 spend 0 days 00:04:00
customer No. 2, at day 6 spend 0 days 00:01:00
customer No. 3, at day 2 spend 0 days 00:02:00
customer No. 3, at day 3 spend 0 days 00:07:00
customer No. 3, at day 4 spend 0 days 00:11:00
customer No. 3, at day 5 spend 0 days 00:25:00
customer No. 3, at day 6 spend 0 days 00:11:00
customer No. 4, at day 2 spend 0 days 00:04:00
customer No. 4, at day 3 spend 0 days 00:06:00
customer No. 4, at day 4 spend 0 days 00:03:00
customer No. 4, at day 5 spend 0 days 00:01:00
customer No. 4, at day 6 spend 0 days 00:01:00
customer No. 5, at day 2 spend 0 days 00:01:00
customer No. 

In [39]:
for i in df['customer_no'].unique():
    df_sub = df.loc[df['customer_no']==i]
    time = df_sub.index.unique().max() - df_sub.index.unique().min() 
    print(f'customer No. {i} spend {time}')  

customer No. 1 spend 0 days 00:02:00
customer No. 2 spend 0 days 00:03:00
customer No. 3 spend 0 days 00:02:00
customer No. 4 spend 0 days 00:04:00
customer No. 5 spend 0 days 00:01:00
customer No. 6 spend 0 days 00:25:00
customer No. 7 spend 0 days 00:09:00
customer No. 8 spend 0 days 00:03:00
customer No. 9 spend 0 days 00:07:00
customer No. 14 spend 0 days 00:03:00
customer No. 13 spend 0 days 00:10:00
customer No. 10 spend 0 days 00:02:00
customer No. 11 spend 0 days 00:02:00
customer No. 12 spend 0 days 00:04:00
customer No. 16 spend 0 days 00:04:00
customer No. 15 spend 0 days 00:05:00
customer No. 17 spend 0 days 00:11:00
customer No. 19 spend 0 days 00:08:00
customer No. 20 spend 0 days 00:02:00
customer No. 18 spend 0 days 00:05:00
customer No. 22 spend 0 days 00:04:00
customer No. 21 spend 0 days 00:03:00
customer No. 24 spend 0 days 00:06:00
customer No. 23 spend 0 days 00:01:00
customer No. 29 spend 0 days 00:02:00
customer No. 27 spend 0 days 00:17:00
customer No. 26 spend

5. Calculate the total number of customers in the supermarket over time.

In [40]:
df.groupby(df.index)['customer_no'].count()

timestamp
2019-09-02 07:03:00    2
2019-09-02 07:04:00    6
2019-09-02 07:05:00    4
2019-09-02 07:06:00    8
2019-09-02 07:07:00    4
                      ..
2019-09-06 21:46:00    5
2019-09-06 21:47:00    4
2019-09-06 21:48:00    7
2019-09-06 21:49:00    4
2019-09-06 21:50:00    6
Name: customer_no, Length: 4363, dtype: int64

6. Our business managers think that the first section customers visit follows a different pattern than the following ones. 
- Plot the distribution of customers of their first visited section versus following sections (treat all sections visited after the first as “following”).

In [54]:
df['first'] = 0
df

,customer_no,location,customer_ordered,day,first
timestamp,,,,,
2019-09-02 07:03:00,1,dairy,1,2,0
2019-09-02 07:03:00,2,dairy,2,2,0
2019-09-02 07:04:00,3,dairy,3,2,0
2019-09-02 07:04:00,4,dairy,4,2,0
2019-09-02 07:04:00,5,spices,5,2,0
...,...,...,...,...,...
2019-09-06 21:50:00,1500,dairy,7435,6,0
2019-09-06 21:50:00,1507,checkout,7442,6,0
2019-09-06 21:50:00,1508,checkout,7443,6,0


In [55]:
df.groupby(['customer_no','location']).count()

customer_ordered  day  first
customer_no location                              
1           checkout                 5    5      5
            dairy                    2    2      2
            drinks                   2    2      2
            fruit                    2    2      2
            spices                   1    1      1
...                                ...  ...    ...
1533        dairy                    1    1      1
1534        checkout                 1    1      1
            fruit                    1    1      1
1535        checkout                 1    1      1
            spices                   1    1      1

[7336 rows x 3 columns]

In [56]:
df[df[]]

SyntaxError: invalid syntax (<ipython-input-56-de69c5818405>, line 1)

In [ ]:
df.groupby(['location', 'transition']).count()

In [ ]:
df['transition'] = df['location'].shift(-1)
df

In [ ]:
P = pd.crosstab(df['location'], df['transition'], normalize='index')
P

In [ ]:
for i in df['customer_no'].unique():
    for j in df['day'].unique():
        df_sub = df.loc[(df['customer_no']==i) & (df['day'] ==j)]
        df_sub[['location',"transition"]].iloc[0]
     
        print(f'customer No. {i}, at day {j} spend {time}')    

In [ ]:
df_test = df.loc[(df['customer_no']==1) & (df['day'] == 2)]
df_test

In [ ]:
df_test[['location',"transition"]].iloc[0]

In [ ]:
type(d)